# 금보원 데이터챌린지_계획서 제출 코드

## 1. 데이터셋 및 라이브러리 불러오기

In [1]:
import pandas as pd

In [2]:
samsung = pd.read_csv(r"E:\금보원_데이터챌린지대회\삼성카드_data.csv",encoding='cp949')
insurance = pd.read_csv(r"E:\금보원_데이터챌린지대회\보험개발원_data.csv",encoding='cp949')
class_name = pd.read_csv('./카드_대중소분류.csv',encoding='cp949')

### 데이터셋 concat

In [3]:
df= pd.concat([samsung, insurance], axis=1)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 405 entries, 기준년도 to 자동차보험가입경력
dtypes: float64(3), int64(391), object(11)
memory usage: 31.8+ KB


In [4]:
df.fillna(0,inplace=True)

## 생명 보험 1, 장기 보험 2, 자동차 보험 3

In [4]:
df["보험료"]=df["보험료"].apply(lambda x: x.strip().replace(",","")).astype(int)
df["지급금액"]=df["지급금액"].apply(lambda x: x.strip().replace(",","")).astype(int)

df['생명보험사고율']=df["사고건수"]/df["계약건수"]
df['장기보험사고율']=df["지급금액"]/df["보험료"]
df['자동차보험사고율']= df["지급금액"]/df["보험료"]

In [31]:
def grade_acc(row):
    if row['보험종류'] == 1:
        ratio=row["생명보험사고율"]
    
    else:
        ratio=row["장기보험사고율"]
        
    if ratio <= 0.2:
            return 0
    elif ratio <= 0.4:
        return 1

    elif ratio <= 0.6:
        return 2

    elif ratio <= 0.8:
        return 3  
    else:
        return 4 

    
    return 4

In [26]:
df["보험_사고율_등급"]= df.apply(grade_acc, axis=1)

In [32]:
print(df[df['보험종류']==1]['생명보험사고율'].value_counts())
print(df[df['보험종류']==2]['장기보험사고율'].value_counts())
print(df[df['보험종류']==3]['자동차보험사고율'].value_counts())

0.0    4
1.0    1
Name: 생명보험사고율, dtype: int64
0.0     1
40.0    1
Name: 장기보험사고율, dtype: int64
35.00    1
43.75    1
0.00     1
Name: 자동차보험사고율, dtype: int64


In [78]:
df[['보험종류', '보험_사고율_등급', '사고건수', '계약건수', '생명보험사고율']]

,보험종류,보험_사고율_등급,사고건수,계약건수,생명보험사고율
0,1,0,0,0.5,0.00
1,1,0,0,1.0,0.00
2,1,4,1,1.0,1.00
3,1,0,0,1.0,0.00
4,2,0,0,2.0,0.00
5,2,4,2,1.0,2.00
6,3,4,1,0.8,1.25
7,3,4,1,1.0,1.00
8,3,0,0,1.0,0.00
9,1,0,0,1.0,0.00


사고율 feature 추가

## 2. 데이터 분석 

## 3. feature 생성

- 평균 매출 금액 = 매출 금액 합 / 매출 금액 건수

0. feature 대분류 dictionary 만들기

In [6]:
big_group = set(class_name['대'])

class_dic = {}

for big in big_group:
    class_dic[big] = {}
    
    middle_group = set(class_name[class_name['대']==big]['중'])
    
    for middle in middle_group:
        small_group = set(class_name[(class_name['대']==big) & (class_name['중']==middle)]['소'])
        class_dic[big][middle] = small_group

print(class_dic)

{'쇼핑': {'문구사무용품': {'문구사무용품'}, '면세점': {'면세점'}, '아울렛': {'아울렛'}, '마트/할인점': {'대형마트/할인점', '마트'}, '스포츠의류용품': {'스포츠의류용품'}, '가전': {'가전'}, '쇼핑기타': {'방문판매'}, '패션잡화': {'패션잡화'}, '백화점': {'백화점'}, '의류': {'의류'}, '자': {'주류판매점'}, '화장품': {'화장품', '미용용품'}, '온라인쇼핑': {'오픈마켓', '종합몰', 'PG_일반인증', '기타쇼핑몰', '홈쇼핑'}, '상품권': {'상품권'}, '편의점': {'연금_복지매장', '편의점'}}, '생활': {'공과금': {'국세', '공공요금', '지방세'}, '가례': {'가례'}, '정기결제': {'4대보험', '전기', '통신비', '도시가스', '관리비'}, '보험': {'생명보험', '손해보험'}, '주생활관리': {'주생활관리'}, '인테리어': {'인테리어'}}, '유흥': {'유흥': {'유흥'}}, '여행': {'교통': {'고속버스', '철도', '여객선', '항공', '렌터카', '고속도로휴게실'}, '숙박': {'여관', '호텔', '콘도', '캠프장'}, '여행상품': {'관광여행사'}}, '자동차': {'주차': {'주차장'}, '차량구매': {'중고차', '수입차', '국산차', '오토바이'}, '대중교통': {'통행료', '택시', '버스/지하철'}, '주유': {'주유'}, '차량관리': {'차량관리'}}, '여가': {'동물병원': {'동물병원'}, '놀이공원': {'놀이공원'}, '취미': {'취미시설', '취미용품'}, '관람': {'관람_기타', '영화'}, '스포츠시설': {'스포츠시설'}, '골프장': {'골프장', '실내외골프연습장'}}, '건강': {'병원': {'이비인후과', '양방종합', '치과', '안과', '피부비뇨기과', '한방종합', '내과', '비뇨기과', '보건소', '기타외과', '한의원', '산부인과_조산

1) 대분류별 데이터셋 생성

In [12]:
big_group

{'건강', '교육', '생활', '쇼핑', '여가', '여행', '요식', '유흥', '자동차'}

In [7]:
big_list = list(big_group)#["요식","자동차","교육","건강","여가","생활","유흥","쇼핑","여행"]

In [8]:
alpha = ["A","B","C","D","E","F","G"]

In [9]:
data = df[['기준년도', 'AGE', 'GENDER', 'JOB', 'INCOME', 'HOM_MGPO', 'HOM_SGG',
       'OFFI_MGPO', 'OFFI_SGG', 'ONEPER_GD_C', 'EST_LFSTG', 'UMRD_SCORE',
       'WEDD_EXP_SCORE', 'CHDB_EXP_SCORE', 'PSCH_OCH_SCORE', 'ESTUD_OCH_SCORE',
       'MHSTUD_OCH_SCORE', 'CSTUD_OCH_SCORE', 'OCH_WEDD_EXP_SCORE',
       'HOUSEWF_SCORE', 'ONL_PRSN', 'ERADP_TNDC', 'PRMM_CSM_TNDC', 'MDLV_PRSN',
       'PET_PRSN', 'ODS_ACTI_INRT_TNDC', 'FRN_TRV_INRT_TNDC',
       'INTERI_SLNG_TNDC', 'INTERI_REMDL_TNDC', 'HL_MNGT_INRT_TNDC']].copy()

In [10]:
df.loc[:,'총사용건수_합'] = (df['총사용건수_A'].values+df['총사용건수_B'].values+df['총사용건수_C'].values+df['총사용건수_D'].values+df['총사용건수_E'].values)
df.loc[:,'총사용금액_합'] = (df['총사용금액_A'].values+df['총사용금액_B'].values+df['총사용금액_C'].values+df['총사용금액_D'].values+df['총사용금액_E'].values)
df.loc[:,'총_평균금액_합'] = df['총사용금액_합']/df['총사용건수_합']

In [34]:
df.head(1)

,기준년도,AGE,GENDER,JOB,INCOME,HOM_MGPO,HOM_SGG,OFFI_MGPO,OFFI_SGG,ONEPER_GD_C,...,보장내용,계약건수,보험료,사고건수,지급금액,자동차국외산구분코드,자동차보험가입경력,생명보험사고율,장기보험사고율,자동차보험사고율
0,2018,A,M,A,A,Seoul,Jung-gu,Seoul,Dongdaemun-gu,5,...,1,0.5,13000,0,0,NaN,NaN,0.0,0.0,0.0


In [35]:
df.columns[:100]

Index(['기준년도', 'AGE', 'GENDER', 'JOB', 'INCOME', 'HOM_MGPO', 'HOM_SGG',
       'OFFI_MGPO', 'OFFI_SGG', 'ONEPER_GD_C', 'EST_LFSTG', 'UMRD_SCORE',
       'WEDD_EXP_SCORE', 'CHDB_EXP_SCORE', 'PSCH_OCH_SCORE', 'ESTUD_OCH_SCORE',
       'MHSTUD_OCH_SCORE', 'CSTUD_OCH_SCORE', 'OCH_WEDD_EXP_SCORE',
       'HOUSEWF_SCORE', 'ONL_PRSN', 'ERADP_TNDC', 'PRMM_CSM_TNDC', 'MDLV_PRSN',
       'PET_PRSN', 'ODS_ACTI_INRT_TNDC', 'FRN_TRV_INRT_TNDC',
       'INTERI_SLNG_TNDC', 'INTERI_REMDL_TNDC', 'HL_MNGT_INRT_TNDC', '총사용건수_A',
       '총사용건수_B', '총사용건수_C', '총사용건수_D', '총사용건수_E', '총사용건수_F', '총사용건수_G',
       '매출건수합계_요식_A', '매출건수합계_요식_B', '매출건수합계_요식_C', '매출건수합계_요식_D',
       '매출건수합계_요식_E', '매출건수합계_요식_F', '매출건수합계_요식_G', '매출건수합계_자동차_A',
       '매출건수합계_자동차_B', '매출건수합계_자동차_C', '매출건수합계_자동차_D', '매출건수합계_자동차_E',
       '매출건수합계_자동차_F', '매출건수합계_자동차_G', '매출건수합계_교육_A', '매출건수합계_교육_B',
       '매출건수합계_교육_C', '매출건수합계_교육_D', '매출건수합계_교육_E', '매출건수합계_교육_F',
       '매출건수합계_교육_G', '매출건수합계_건강_A', '매출건수합계_건강_B', '매출건수합계_건강_C',
  

In [12]:
df_columns = df.columns

for big in class_dic.keys():
    df.loc[:,big+'_총사용건수_합']=0
    df.loc[:,big+'_총사용금액_합']=0
    for a in alpha:
        df.loc[:,big+'_총사용건수_합'] += df["매출건수합계_"+big+"_"+a].values
        df.loc[:,big+'_총사용금액_합'] += df["매출금액합계_"+big+"_"+a].values
    df.loc[:,big+'_평균사용금액']= df[big+'_총사용금액_합']/df[big+'_총사용건수_합']
    for mid in class_dic[big].keys():
        df.loc[:,mid+'_총사용건수_합']=0
        df.loc[:,mid+'_총사용금액_합']=0
        if mid not in df_columns:
            print(big,'의',mid,'없음')
            continue
        for a in alpha:
            df.loc[:,mid+'_총사용건수_합'] += df["매출건수합계_"+mid+"_"+a].values
            df.loc[:,mid+'_총사용금액_합'] += df["매출금액합계_"+mid+"_"+a].values
        df.loc[:,mid+'_평균사용금액']= df[mid+'_총사용금액_합']/df[mid+'_총사용건수_합']
        for small in class_dic[big][mid]:
            df.loc[:,small+'_총사용건수_합']=0
            df.loc[:,small+'_총사용금액_합']=0
            if small in df_columns:
                for a in alpha:
                    df.loc[:,small+'_총사용건수_합'] += df["매출건수합계_"+small+"_"+a].values
                    df.loc[:,small+'_총사용금액_합'] += df["매출금액합계_"+small+"_"+a].values
                df.loc[:,small+'_평균사용금액']= df[small+'_총사용금액_합']/df[small+'_총사용건수_합']
            else:
                print(big,'의',mid,'의 ',small,'없음')
            

In [13]:
for big in class_dic.keys():
    
    if ("매출건수합계_"+big+"_A") not in df_columns:
        print(big,'없음')
        continue
    tmp_count = []
    tmp_money = []
    for a in alpha:
        tmp_count.append("매출건수합계_"+big+"_"+a)
        tmp_money.append("매출금액합계_"+big+"_"+a)
    
    tmp_count_sum = df[tmp_count].sum(axis=1)
    tmp_money_sum = df[tmp_money].sum(axis=1)
    
    total = tmp_money_sum/tmp_count_sum
    df.loc[:,big+'_평균사용금액']= total
    
    for mid in class_dic[big].keys():
        if ("매출건수합계_"+mid+"_A") not in df_columns:
            print(big,'의 ',mid,'없음')
            continue
        tmp_count = []
        tmp_money = []
        for a in alpha:
            tmp_count.append("매출건수합계_"+mid+"_"+a)
            tmp_money.append("매출금액합계_"+mid+"_"+a)
    
        tmp_count_sum = df[tmp_count].sum(axis=1)
        tmp_money_sum = df[tmp_money].sum(axis=1)
        df.loc[:,mid+'_평균사용금액']= total
        
        for small in class_dic[big][mid]:
            if ("매출건수합계_"+small+"_A") not in df_columns:
                print(big,'의 ',mid,'의',small,'없음')
                continue
            tmp_count = []
            tmp_money = []
            for a in alpha:
                tmp_count.append("매출건수합계_"+small+"_"+a)
                tmp_money.append("매출금액합계_"+small+"_"+a)

            tmp_count_sum = df[tmp_count].sum(axis=1)
            tmp_money_sum = df[tmp_money].sum(axis=1)
            df.loc[:,small+'_평균사용금액']= total
    

쇼핑 의  문구사무용품 없음
쇼핑 의  면세점 없음
쇼핑 의  아울렛 없음
쇼핑 의  마트/할인점 없음
쇼핑 의  스포츠의류용품 없음
쇼핑 의  가전 없음
쇼핑 의  쇼핑기타 없음
쇼핑 의  패션잡화 없음
쇼핑 의  백화점 없음
쇼핑 의  의류 없음
쇼핑 의  자 없음
쇼핑 의  화장품 없음
쇼핑 의  온라인쇼핑 의 오픈마켓 없음
쇼핑 의  온라인쇼핑 의 종합몰 없음
쇼핑 의  온라인쇼핑 의 PG_일반인증 없음
쇼핑 의  온라인쇼핑 의 기타쇼핑몰 없음
쇼핑 의  온라인쇼핑 의 홈쇼핑 없음
쇼핑 의  상품권 없음
쇼핑 의  편의점 없음
생활 의  공과금 없음
생활 의  가례 없음
생활 의  정기결제 없음
생활 의  보험 없음
생활 의  주생활관리 없음
생활 의  인테리어 없음
여행 의  교통 없음
여행 의  숙박 없음
여행 의  여행상품 없음
자동차 의  주차 의 주차장 없음
자동차 의  차량구매 의 중고차 없음
자동차 의  차량구매 의 수입차 없음
자동차 의  차량구매 의 국산차 없음
자동차 의  차량구매 의 오토바이 없음
자동차 의  대중교통 의 통행료 없음
자동차 의  대중교통 의 택시 없음
자동차 의  대중교통 의 버스/지하철 없음
자동차 의  차량관리 없음
여가 의  놀이공원 없음
여가 의  취미 없음
여가 의  관람 없음
여가 의  골프장 의 실내외골프연습장 없음
건강 의  병원 의 이비인후과 없음
건강 의  병원 의 양방종합 없음
건강 의  병원 의 치과 없음
건강 의  병원 의 안과 없음
건강 의  병원 의 피부비뇨기과 없음
건강 의  병원 의 한방종합 없음
건강 의  병원 의 내과 없음
건강 의  병원 의 비뇨기과 없음
건강 의  병원 의 보건소 없음
건강 의  병원 의 기타외과 없음
건강 의  병원 의 한의원 없음
건강 의  병원 의 산부인과_조산원 없음
건강 의  병원 의 성형외과 없음
건강 의  병원 의 피부과 없음
건강 의  병원 의 양한방종합 없음
건강 의  병원 의 방사선과 없음
건강 의  뷰티 없음
요식 의  커피/음료 없음
요식 의

In [15]:
big_group

{'건강', '교육', '생활', '쇼핑', '여가', '여행', '요식', '유흥', '자동차'}

In [23]:
df.columns[200:250]

Index(['매출건수합계_요양원_C', '매출건수합계_요양원_D', '매출건수합계_요양원_E', '매출건수합계_요양원_F',
       '매출건수합계_요양원_G', '매출건수합계_의료기기용품_A', '매출건수합계_의료기기용품_B', '매출건수합계_의료기기용품_C',
       '매출건수합계_의료기기용품_D', '매출건수합계_의료기기용품_E', '매출건수합계_의료기기용품_F',
       '매출건수합계_의료기기용품_G', '총사용금액_A', '총사용금액_B', '총사용금액_C', '총사용금액_D',
       '총사용금액_E', '총사용금액_F', '총사용금액_G', '매출금액합계_요식_A', '매출금액합계_요식_B',
       '매출금액합계_요식_C', '매출금액합계_요식_D', '매출금액합계_요식_E', '매출금액합계_요식_F',
       '매출금액합계_요식_G', '매출금액합계_자동차_A', '매출금액합계_자동차_B', '매출금액합계_자동차_C',
       '매출금액합계_자동차_D', '매출금액합계_자동차_E', '매출금액합계_자동차_F', '매출금액합계_자동차_G',
       '매출금액합계_교육_A', '매출금액합계_교육_B', '매출금액합계_교육_C', '매출금액합계_교육_D',
       '매출금액합계_교육_E', '매출금액합계_교육_F', '매출금액합계_교육_G', '매출금액합계_건강_A',
       '매출금액합계_건강_B', '매출금액합계_건강_C', '매출금액합계_건강_D', '매출금액합계_건강_E',
       '매출금액합계_건강_F', '매출금액합계_건강_G', '매출금액합계_여가_A', '매출금액합계_여가_B',
       '매출금액합계_여가_C'],
      dtype='object')

In [17]:
class_dic

{'생활': {'보험': {'생명보험', '손해보험'},
  '인테리어': {'인테리어'},
  '정기결제': {'4대보험', '관리비', '도시가스', '전기', '통신비'},
  '주생활관리': {'주생활관리'},
  '가례': {'가례'},
  '공과금': {'공공요금', '국세', '지방세'}},
 '여행': {'교통': {'고속도로휴게실', '고속버스', '렌터카', '여객선', '철도', '항공'},
  '숙박': {'여관', '캠프장', '콘도', '호텔'},
  '여행상품': {'관광여행사'}},
 '요식': {'커피/음료': {'커피/음료'},
  '제과점': {'제과점_아이스크림점'},
  '음식점': {'간이음식점', '부페', '양식', '일식', '주점', '중식', '패스트푸드점', '한식'},
  '배달앱': {'배달앱'}},
 '자동차': {'차량관리': {'차량관리'},
  '대중교통': {'버스/지하철', '택시', '통행료'},
  '주유': {'주유'},
  '주차': {'주차장'},
  '차량구매': {'국산차', '수입차', '오토바이', '중고차'}},
 '교육': {'서적': {'서적'},
  '교육기타': {'교육기타'},
  '학원': {'기능학원', '문리계학원', '예체능계학원', '외국어학원'},
  '학교': {'대학교', '초_중_고등학교', '특수학교'},
  '유아교육': {'유아교육'}},
 '유흥': {'유흥': {'유흥'}},
 '여가': {'스포츠시설': {'스포츠시설'},
  '동물병원': {'동물병원'},
  '관람': {'관람_기타', '영화'},
  '골프장': {'골프장', '실내외골프연습장'},
  '놀이공원': {'놀이공원'},
  '취미': {'취미시설', '취미용품'}},
 '건강': {'건강보조': {'건강보조식품', '산후조리원', '안경', '양약국', '요양원', '의료기기용품'},
  '뷰티': {'미용관리', '미용실', '이발소'},
  '병원': {'기타외과',
 

### 새로운 feature 생성

출퇴근 간격 생성

In [44]:
map_dict = {}

In [38]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [41]:
import googletrans
translator = googletrans.Translator()


def trans(text):#번역기
    try:
        if not text.encode().isalpha():#한글=>영어
            return translator.translate(text, dest='en').text
        else:#영어=>한글
            return translator.translate(text, dest='ko').text
    except:
        print("번역 에러")

In [43]:
print(trans('seoul seongbukgu'))

번역 에러
None


In [52]:
import os
import sys
import urllib.request

client_id = "dfegGXhsPL7vfqj2yg3i"
client_secret = "HXjgTp_aS5"
encText = urllib.parse.quote('seoul seongbukgu')
data = "source=en&target=ko&text="+encText
url = "https://openapi.naver.com/v1/papago/n2mt"
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request,data=data.encode('utf-8'))
rescode = response.getcode()

if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("error code"+rescode)
    


{"message":{"result":{"srcLangType":"en","tarLangType":"ko","translatedText":"서울성북구","engineType":"N2MT"},"@type":"response","@service":"naverservice.nmt.proxy","@version":"1.0.0"}}


In [89]:
for row in df.iterrows():
    home = row[1][5]+" "+row[1][6]
    office = row[1][7]+" "+row[1][8]
    if home not in map_dict:
        map_dict[home]=[]
    if office not in map_dict:
        map_dict[office]=[]

In [54]:
driver = webdriver.Chrome('chromedriver')
driver.get('https://map.naver.com/v5/directions/-/-/-/?c=14139078.9231079,4509745.6215210,15,0,0,0,dh')
#네이버 길찾기 주소

#driver.find_element_by_xpath('//*[@id="intro_popup_close"]/span').click()

delay = 3
driver.implicitly_wait(delay) # 3초 기달림 


driver.find_element_by_id('directionStart0').send_keys('서울 성북구')
driver.find_element_by_id("directionStart0").send_keys(Keys.RETURN) # 엔터 

try:
    driver.find_element_by_xpath('//*[@id="container"]/div[1]/shrinkable-layout/directions-layout/directions-result/directions-search-list/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place/div').click()

except:
    driver.find_element_by_id('directionGoal').send_keys('서울 동작구')
    driver.find_element_by_id('directionGoal').send_keys(Keys.RETURN)

try :
    #driver.find_element_by_xpath('//*[@id="container"]/div[1]/shrinkable-layout/directions-layout/directions-result/div/directions-search/div[2]/button[2]').click()
    driver.find_element_by_class_name('btn.btn_direction.active').click()
except:
    print('1')

AttributeError: 'str' object has no attribute 'capabilities'

In [100]:
map_dict

{'Seoul Jung-gu': [],
 'Seoul Dongdaemun-gu': [],
 'Seoul Seocho-gu': [],
 'Seoul Guro-gu': [],
 'Seoul Songpa-gu': [],
 'Gyeonggi Ansan-si danwon-gu': [],
 'Seoul Yeongdeungpo-gu': [],
 'Gyeonggi Seongnam-si bundang-gu': [],
 'Gyeonggi Namyangju-si': [],
 'Gyeonggi Goyang-si ilsandong-gu': [],
 'Gyeonggi Siheung-si': [],
 'Incheon Michuhol-gu': [],
 'Gyeonggi Uijeongbu-si': [],
 'Seoul Gwangjin-gu': [],
 'Seoul Yangcheon-gu': [],
 'Incheon Namdong-gu': [],
 'Incheon Bupyeong-gu': [],
 'Incheon Seo-gu': []}

In [23]:
def result_location(map_dict):
    for i in map_dict:
        api_json = get_location(i)
        print(api_json)
        if api_json['documents']:
            address = api_json['documents'][0]['address']
            map_dict[i]= [address['x'],address['y']]     
        time.sleep(2)
    return map_dict

In [24]:
temp = {'서울특별시 종로구 종로1길 36, 대림빌딩 (수송동)'}
map_dict_ = result_location(temp)

{'documents': [{'address': {'address_name': '서울 종로구 수송동 146-12', 'b_code': '1111012400', 'h_code': '1111061500', 'main_address_no': '146', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '종로구', 'region_3depth_h_name': '종로1.2.3.4가동', 'region_3depth_name': '수송동', 'sub_address_no': '12', 'x': '126.978988255925', 'y': '37.5735051436739'}, 'address_name': '서울 종로구 종로1길 36', 'address_type': 'ROAD_ADDR', 'road_address': {'address_name': '서울 종로구 종로1길 36', 'building_name': '종로구청', 'main_building_no': '36', 'region_1depth_name': '서울', 'region_2depth_name': '종로구', 'region_3depth_name': '수송동', 'road_name': '종로1길', 'sub_building_no': '', 'underground_yn': 'N', 'x': '126.978988255925', 'y': '37.5735051436739', 'zone_no': '03152'}, 'x': '126.978988255925', 'y': '37.5735051436739'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}


TypeError: 'set' object does not support item assignment

## 2. 등급 부여하기
- 등급은 값이 높을 수록 평균 사용금액이 높은 것을 의미한다.

In [14]:
def grade(bin_number,series):
    min_ = min(series.values)
    max_ = max(series.values)
    bin_size = (max_-min_)/bin_number
    grade_list = series.values
    # 등급 생성
    grade_list = []
    data = min_
    for i in range(bin_number+1):
        grade_list.append(data)
        data+= bin_size
    return grade_list

In [15]:
def grade_apply(x,columns_list):
    for i in range(len(columns_list)-1) :
        if x<=columns_list[i+1]:
            return i+1
    return len(columns_list)-1

In [90]:
# grade_list 전역변수로 지정 

In [16]:
columns_grade_list = ["요식_평균사용금액", '자동차_평균사용금액', '교육_평균사용금액', '건강_평균사용금액', '여가_평균사용금액',
       '생활_평균사용금액', '유흥_평균사용금액', '쇼핑_평균사용금액','UMRD_SCORE',
       'WEDD_EXP_SCORE', 'CHDB_EXP_SCORE', 'PSCH_OCH_SCORE', 'ESTUD_OCH_SCORE',
       'MHSTUD_OCH_SCORE', 'CSTUD_OCH_SCORE', 'OCH_WEDD_EXP_SCORE',
       'HOUSEWF_SCORE', 'ONL_PRSN', 'ERADP_TNDC', 'PRMM_CSM_TNDC', 'MDLV_PRSN',
       'PET_PRSN', 'ODS_ACTI_INRT_TNDC', 'FRN_TRV_INRT_TNDC',
       'INTERI_SLNG_TNDC', 'INTERI_REMDL_TNDC', 'HL_MNGT_INRT_TNDC',]
 

In [17]:
#보험 사고율 등급 
grade_list_1 = grade(5,df['생명보험사고율'])
grade_list_2 = grade(5,df['장기보험사고율'])
grade_list_3 = grade(5,df['자동차보험사고율'])

In [18]:
for c in columns_grade_list:
    grade_list = grade(10,df[c]) # 분류별 범위 리스트 생성
    df.loc[:,c+"등급"]=df[c].apply(grade_apply,args=(grade_list,)) # apply 작성

In [19]:
df.loc[:,"생명보험사고율_등급"]=df["생명보험사고율"].apply(grade_apply,args=(grade_list_1,)) # apply 작성
df.loc[:,"장기보험사고율_등급"]=df["장기보험사고율"].apply(grade_apply,args=(grade_list_2,)) # apply 작성
df.loc[:,"자동차보험사고율_등급"]=df["자동차보험사고율"].apply(grade_apply,args=(grade_list_3,)) # apply 작성

# 3. 대분류 그래프 생성

In [ ]:
# 데이터셋 train test나눌꺼면 나누기 
X = []
y = []
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

## 1)사고율 별 그룹 생성

data drop

In [20]:
feature_list = ['AGE', 'GENDER', 'JOB', 'INCOME','요식_평균사용금액등급',
       '자동차_평균사용금액등급', '교육_평균사용금액등급', '건강_평균사용금액등급', '여가_평균사용금액등급',
       '생활_평균사용금액등급', '유흥_평균사용금액등급', '쇼핑_평균사용금액등급', 'UMRD_SCORE등급',
       'WEDD_EXP_SCORE등급', 'CHDB_EXP_SCORE등급', 'PSCH_OCH_SCORE등급',
       'ESTUD_OCH_SCORE등급', 'MHSTUD_OCH_SCORE등급', 'CSTUD_OCH_SCORE등급',
       'OCH_WEDD_EXP_SCORE등급', 'HOUSEWF_SCORE등급', 'ONL_PRSN등급', 'ERADP_TNDC등급',
       'PRMM_CSM_TNDC등급', 'MDLV_PRSN등급', 'PET_PRSN등급', 'ODS_ACTI_INRT_TNDC등급',
       'FRN_TRV_INRT_TNDC등급', 'INTERI_SLNG_TNDC등급', 'INTERI_REMDL_TNDC등급',
       'HL_MNGT_INRT_TNDC등급']

In [42]:
df[['AGE','GENDER']]

,AGE,GENDER
0,A,M
1,B,F
2,C,M
3,D,F
4,E,M
5,F,F
6,G,M
7,H,F
8,I,M
9,B,F


In [43]:
df.columns

Index(['기준년도', 'AGE', 'GENDER', 'JOB', 'INCOME', 'HOM_MGPO', 'HOM_SGG',
       'OFFI_MGPO', 'OFFI_SGG', 'ONEPER_GD_C',
       ...
       '산후조리원_평균사용금액', '안경_평균사용금액', '건강보조식품_평균사용금액', '요양원_평균사용금액', '병원_평균사용금액',
       '쇼핑_평균사용금액', '온라인쇼핑_평균사용금액', '생명보험사고율_등급', '장기보험사고율_등급', '자동차보험사고율_등급'],
      dtype='object', length=439)

In [44]:
tmp =['AGE', 'GENDER', 'JOB', 'INCOME','요식_평균사용금액등급',
       '자동차_평균사용금액등급', '교육_평균사용금액등급', '건강_평균사용금액등급', '여가_평균사용금액등급',
       '생활_평균사용금액등급', '유흥_평균사용금액등급', '쇼핑_평균사용금액등급', 'UMRD_SCORE등급',
       'WEDD_EXP_SCORE등급', 'CHDB_EXP_SCORE등급', 'PSCH_OCH_SCORE등급',
       'ESTUD_OCH_SCORE등급', 'MHSTUD_OCH_SCORE등급', 'CSTUD_OCH_SCORE등급',
       'OCH_WEDD_EXP_SCORE등급', 'HOUSEWF_SCORE등급', 'ONL_PRSN등급', 'ERADP_TNDC등급',
       'PRMM_CSM_TNDC등급', 'MDLV_PRSN등급', 'PET_PRSN등급', 'ODS_ACTI_INRT_TNDC등급',
       'FRN_TRV_INRT_TNDC등급', 'INTERI_SLNG_TNDC등급', 'INTERI_REMDL_TNDC등급',
       'HL_MNGT_INRT_TNDC등급']

In [46]:
for t in tmp:
    if t not in df_columns:
        print(t)
        

요식_평균사용금액등급
자동차_평균사용금액등급
교육_평균사용금액등급
건강_평균사용금액등급
여가_평균사용금액등급
생활_평균사용금액등급
유흥_평균사용금액등급
쇼핑_평균사용금액등급
UMRD_SCORE등급
WEDD_EXP_SCORE등급
CHDB_EXP_SCORE등급
PSCH_OCH_SCORE등급
ESTUD_OCH_SCORE등급
MHSTUD_OCH_SCORE등급
CSTUD_OCH_SCORE등급
OCH_WEDD_EXP_SCORE등급
HOUSEWF_SCORE등급
ONL_PRSN등급
ERADP_TNDC등급
PRMM_CSM_TNDC등급
MDLV_PRSN등급
PET_PRSN등급
ODS_ACTI_INRT_TNDC등급
FRN_TRV_INRT_TNDC등급
INTERI_SLNG_TNDC등급
INTERI_REMDL_TNDC등급
HL_MNGT_INRT_TNDC등급


보험별로 나누는거는 코드는 짜도 지금 적용하면 데이터가 없어서 pass 

## 2) 그래프 생성

In [23]:
def filter_graph(acc_dic, acc_data, min_count, max_group, feature_list):
    for column in feature_list:
        value_counts = acc_data[column].value_counts()
        
        value_counts = {key: value for key, value in value_counts.items() if value > min_count}
        
        if len(value_counts) > max_group:
            continue
        
        for value, count in value_counts.items():
            acc_dic[f'{column}_{value}'] = set(acc_data[acc_data[column] == value]['index'])

In [24]:
def merge_big_graph(acc_dic, acc_data, max_group):
    temp = dict()
    c_list = list()
    
    group_list = acc_dic.keys()
    
    for group in group_list:
        big_set = set()
        big_group_data = acc_data[acc_data['index'].isin(acc_dic[group])]
        
        for column in feature_list:
            value_counts = acc_data[column].value_counts()

            if len(value_counts) > max_group:
                continue
            
            for value, count in value_counts.items():
                c_list.append(f'{column}_{value}')
            
            for big in class_dic.keys():
                if big in column:
                    big_set.add(big)
                    
        group_result = '-'.join(c_list)
        temp[group_result] = (big_set, big_group_data['index'])
        
    acc_dic = temp

In [60]:
biggroup_dic[j] = dict()
acc_dic = biggroup_dic[j]

# 대분류 필터링
filter_graph(acc_dic, acc_data, min_count, max_group, feature_list) # 사고율 별 1차 필터링
merge_big_graph(acc_dic, acc_data, max_group) # 대분류 2차 필터링

NameError: name 'biggroup_dic' is not defined

In [62]:
num_acc_group

5

In [31]:
df = df.reset_index()

In [33]:
graph = {}
num_acc_group = 5
max_group = 2
min_count = 0

for i in range(1,4):
    insurance_data = df[df['보험종류']==i]
    
    graph[i] = dict()
    biggroup_dic = graph[i]
    for j in range(1, num_acc_group+1): # 사고율 별 계산
        if j==1 :
            name = "생명보험사고율_등급"
            acc_data = insurance_data[insurance_data['생명보험사고율_등급']==j]
        elif j==2 :
            name = "장기보험사고율_등급"
            acc_data = insurance_data[insurance_data['장기보험사고율_등급']==j]
        else:
            name = "자동차보험사고율_등급"
            acc_data = insurance_data[insurance_data['자동차보험사고율_등급']==j]
            
        
        
        biggroup_dic[j] = dict()
        acc_dic = biggroup_dic[j]
        
        # 대분류 필터링
        filter_graph(acc_dic, acc_data, min_count, max_group, feature_list) # 사고율 별 1차 필터링
        merge_big_graph(acc_dic, acc_data, max_group) # 대분류 2차 필터링
        
        # 중분류 필터링
        for big_feature in acc_dic.keys(): # 대분류 별 진행
            feature_list = []
            print(big_list)
            big_list = acc_dic[big_feature][0] # 대분류 리스트
            mid_data = acc_data[acc_data['index'].isin(acc_dic[big_feature][1])] # 해당 대분류에 해당하는 데이터
            
            mid_dic = dict()
            big_graph = acc_dic[big_feature]
            
            for b in big_list:
                feature_list.append(i for i in class_dic[b].keys())
            
            # 중분류 필터링
            filter_graph(acc_dic, acc_data, min_count, max_group, feature_list) # 중분류 1차 그래프 생성
            merge_big_graph(acc_dic, acc_data, max_group) # 중분류 2차 필터링
        
            # 소분류도 중분류와 같은 방식으로 진행
            # ....

In [34]:
graph

{1: {1: {}, 2: {}, 3: {}, 4: {}, 5: {}},
 2: {1: {}, 2: {}, 3: {}, 4: {}, 5: {}},
 3: {1: {}, 2: {}, 3: {}, 4: {}, 5: {}}}

## 중복제거

In [35]:
result = dict()

for i in range(1,4): # 보험 별
    biggroup_dic = graph[i]
    feature_set = dict()
        
    for j in range(1, num_acc_group+1): # 사고율 별 feature 집합 구분
        feature_set[j] = set()
        
        for feature in biggroup_dic[j].keys():
            feature_set[j].add(feature)
        
    for now in range(len(feature_set)):
        temp = feature_set[now]
        for other in range(len(feature_set)):
            if now == other:
                continue
            temp = temp.difference(feature_set[other])
    
        result[now] = temp


KeyError: 0

## 3) 검증
- 모델: randomforest 

In [ ]:
for i in range(1,4): # 보험 별
    biggroup_dic = graph[i]
    
    for j in range(1, num_acc_group+1): # 사고율 별 F1-Score 계산 -> 높은 사고율만 할꺼니까 j 수정해야함. 
        answer = set(df[(df['보험종류'] == i) & (df['보험_사고율_등급'] == j)]['index'])
        predict = set()
        
        acc_graph = biggroup_dic[j]
        feature_set = set(acc_graph.keys())
        
        for feature_join in feature_set: # 각 조건 별 해당하는 index 추출 , 일단 나는 feature_set이 
            f_dict = dict()
            feature_pair = feature_join.split('-')
            
            for f in feature_pair:
                feature = f.split('_')
                f_dict[feature[0]] = feature[1]
                
            df_temp = df
            for key, value in f_dict.items():
                df_temp = df_temp[df_temp[key] == value]
            
            predict &= set(df_temp['index'])
                
        TP = len(answer & predict)
        FP = len(predict - answer)
        FN = len(answer - predict)
        
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f1_score = 2 * (precision * recall) / (precision + recall)
    
        print(f'{i}보험 {j}사고율에 해당하는 F1-Score : ', f1_score)

1. graph signature자료구조 
2. 각 df itterrow에 대해서 일치하는지 체크
-> 각 string으로 만들고 하는게 나을지 ? in으로해서? 
3. 일치하면 label list에 1 로 채우기

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
for i in range(1,4): # 보험 별
    biggroup_dic = graph[i]
    
    for j in range(1, num_acc_group+1): # 사고율 별 F1-Score 계산 -> 높은 사고율만 이니까 j 수정해야함. 
        answer = set(df[(df['보험종류'] == i) & (df['보험_사고율_등급'] == j)]['index'])
        predict = []
        
        acc_graph = biggroup_dic[j]
        feature_set = set(acc_graph.keys())#중복제거 위에서 한거 아니였나욥? 
        check = False
        for row in df.iterrows():
            
            for feature_join in feature_set: # 각 조건 별 해당하는 index 추출
                
                feature_pair = feature_join.split('-')
                
                for f in feature_pair:
                    feature = f.split('_')
                    if row[feature[0]]!=feature[1]:
                        check = False
                        break
                    else: 
                        check=True
                if check==True:
                    predict.append(1)
                    break
            if check==False:
                predict.append(0)
        
                

        TP = len(answer & predict)
        FP = len(predict - answer)
        FN = len(answer - predict)
        
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f1_score = 2 * (precision * recall) / (precision + recall)
        
        print(f'{i}보험 {j}사고율에 해당하는 F1-Score : ', f1_score)
        print('TP',TP)
        print('FP',FP)
        print('FN',FN)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, plot_confusion_matrix

In [ ]:
for i in range(1,4): # 보험 별
    biggroup_dic = graph[i]
    
    for j in range(1, num_acc_group+1): # 사고율 별 F1-Score 계산 -> 높은 사고율만 이니까 j 수정해야함. 
        answer = set(df[(df['보험종류'] == i) & (df['보험_사고율_등급'] == j)]['index'])
        
        acc_graph = biggroup_dic[j]
        feature_set = set(acc_graph.keys())#중복제거 위에서 한거 아니였나욥? 
        
        feature_model = []
        
            
        for feature_join in feature_set: # 각 조건 별 해당하는 index 추출

            feature_pair = feature_join.split('-') # 특징 조각들 

            for f in feature_pair:
                feature = f.split('_')
                feature_model.append(feature[0])
                
        rf = RandomForestClassifier(n_estimators=100, max_depth=5,random_state=0)
        trainX = train_x[feature_model]
        rf.fit(trainX,train_y)
        
        testX = testx[feature_model]
        pre = rf.predict(testX)
        
        print(accuracy_score(test_y,pre))
        print(confusion_matrix(test_y, pre)

## 4) 평가 코드

In [ ]:
test_df = 'temp'

# test도 같은 과정 거쳐서 생성되는 특징 set이 유사한지 살펴보기.. ?

# 전처리
df["보험료"]=df["보험료"].apply(lambda x: x.strip().replace(",","")).astype(int)
df["지급금액"]=df["지급금액"].apply(lambda x: x.strip().replace(",","")).astype(int)

df['생명보험사고율']=df["사고건수"]/df["계약건수"]
df['장기보험사고율']=df["지급금액"]/df["보험료"]
df['자동차보험사고율']= df["지급금액"]/df["보험료"]

df.loc[:,'총사용건수_합'] = (df['총사용건수_A'].values+df['총사용건수_B'].values+df['총사용건수_C'].values+df['총사용건수_D'].values+df['총사용건수_E'].values)
df.loc[:,'총사용금액_합'] = (df['총사용금액_A'].values+df['총사용금액_B'].values+df['총사용금액_C'].values+df['총사용금액_D'].values+df['총사용금액_E'].values)
df.loc[:,'총_평균금액_합'] = df['총사용금액_합']/df['총사용건수_합']


# 등급 생성
# graph 생성 및 print 

In [ ]:
for big in class_dic.keys():
    
    if ("매출건수합계_"+big+"_A") not in df_columns:
        print(big,'없음')
        continue
    tmp_count = []
    tmp_money = []
    for a in alpha:
        tmp_count.append("매출건수합계_"+big+"_"+a)
        tmp_money.append("매출금액합계_"+big+"_"+a)
    
    tmp_count_sum = df[tmp_count].sum(axis=1)
    tmp_money_sum = df[tmp_money].sum(axis=1)
    
    total = tmp_money_sum/tmp_count_sum
    df.loc[:,big+'_평균사용금액']= total
    
    for mid in class_dic[big].keys():
        if ("매출건수합계_"+mid+"_A") not in df_columns:
            print(big,'의 ',mid,'없음')
            continue
        tmp_count = []
        tmp_money = []
        for a in alpha:
            tmp_count.append("매출건수합계_"+mid+"_"+a)
            tmp_money.append("매출금액합계_"+mid+"_"+a)
    
        tmp_count_sum = df[tmp_count].sum(axis=1)
        tmp_money_sum = df[tmp_money].sum(axis=1)
        df.loc[:,mid+'_평균사용금액']= total
        
        for small in class_dic[big][mid]:
            if ("매출건수합계_"+small+"_A") not in df_columns:
                print(big,'의 ',mid,'의',small,'없음')
                continue
            tmp_count = []
            tmp_money = []
            for a in alpha:
                tmp_count.append("매출건수합계_"+small+"_"+a)
                tmp_money.append("매출금액합계_"+small+"_"+a)

            tmp_count_sum = df[tmp_count].sum(axis=1)
            tmp_money_sum = df[tmp_money].sum(axis=1)
            df.loc[:,small+'_평균사용금액']= total
    

In [ ]:
df.loc[:,"생명보험사고율_등급"]=df["생명보험사고율"].apply(grade_apply,args=(grade_list_1,)) # apply 작성
df.loc[:,"장기보험사고율_등급"]=df["장기보험사고율"].apply(grade_apply,args=(grade_list_2,)) # apply 작성
df.loc[:,"자동차보험사고율_등급"]=df["자동차보험사고율"].apply(grade_apply,args=(grade_list_3,)) # apply 작성

In [ ]:
graph_test = {}
num_acc_group = 5
max_group = 2
min_count = 0

for i in range(1,4):
    insurance_data = df[df['보험종류']==i]
    
    graph[i] = dict()
    biggroup_dic = graph[i]
    for j in range(1, num_acc_group+1): # 사고율 별 계산
        if j==1 :
            name = "생명보험사고율_등급"
            acc_data = insurance_data[insurance_data['생명보험사고율_등급']==j]
        elif j==2 :
            name = "장기보험사고율_등급"
            acc_data = insurance_data[insurance_data['장기보험사고율_등급']==j]
        else:
            name = "자동차보험사고율_등급"
            acc_data = insurance_data[insurance_data['자동차보험사고율_등급']==j]
            
        
        
        biggroup_dic[j] = dict()
        acc_dic = biggroup_dic[j]
        
        # 대분류 필터링
        filter_graph(acc_dic, acc_data, min_count, max_group, feature_list) # 사고율 별 1차 필터링
        merge_big_graph(acc_dic, acc_data, max_group) # 대분류 2차 필터링
        
        # 중분류 필터링
        for big_feature in acc_dic.keys(): # 대분류 별 진행
            feature_list = []
            print(big_list)
            big_list = acc_dic[big_feature][0] # 대분류 리스트
            mid_data = acc_data[acc_data['index'].isin(acc_dic[big_feature][1])] # 해당 대분류에 해당하는 데이터
            
            mid_dic = dict()
            big_graph = acc_dic[big_feature]
            
            for b in big_list:
                feature_list.append(i for i in class_dic[b].keys())
            
            # 중분류 필터링
            filter_graph(acc_dic, acc_data, min_count, max_group, feature_list) # 중분류 1차 그래프 생성
            merge_big_graph(acc_dic, acc_data, max_group) # 중분류 2차 필터링
        
            # 소분류도 중분류와 같은 방식으로 진행
            # ....

## 5) 그래프 그리기

In [329]:
import networkx as nx
import matplotlib.pyplot as plt

def draw_graph(graph, ensurance, file_name): # 임시 코드 (현재 데이터로 그래프 생성 불가)
    G = nx.DiGraph()

    def add_edges(graph, node, parent_node=None):
        if parent_node:
            G.add_edge(parent_node, node)
        for child_node, value in graph[node].items():
            G.add_edge(node, child_node)
            add_edges(graph[node], child_node, node)

    add_edges(graph, list(graph.keys())[0])

    pos = nx.spring_layout(G)
    nx.draw_networkx(G, pos, with_labels=True, node_color='lightblue', node_size=500, \
                     font_size=12, edge_color='gray',\
                     width=1.0, arrowsize=15, arrowstyle='->')

    plt.axis('off')
    plt.save(f'./figure/{ensurance}/{file_name}_graph.jpeg')
    plt.show()

In [ ]:
for i in range(1,4): # 보험 별 그래프 생성
    biggroup_dic = graph[i]
    
    for j in range(1, num_acc_group+1): # 사고율 별 그래프 생성
        acc_graph = biggroup_dic[j]
        draw_graph(acc_graph, i, j) # 폴더이름은 추후 변경